# Heidi Hendrickson's blsa protein prep_leap_ions workflow

### See: https://pubs.acs.org/doi/abs/10.1021/acsbiomaterials.6b00447

In [ ]:
setupAmberPaths

In [ ]:
source "leaprc.protein.ff14SB"

In [ ]:
source leaprc.water.tip3p

In [ ]:
source leaprc.gaff

Scan the PDB file and generate a summary that will help us convert it to an AMBER PDB file.

In [ ]:
scan = scanPdb data/bsla/4bhu.pdb

Only use the C chain

In [ ]:
scanSelectChainIds scan { :C }

Check if the current filters leave us with an AMBER compatible PDB file.

In [ ]:
scanAmberCheck scan

In [ ]:
scanIgnoreResidues scan { :HOH :CL }

Rename residues MLY->LYS and MSE->MET

In [ ]:
scanRenameResidues scan { { :MLY :LYS } { :MSE :MET } }

In [ ]:
scanAmberCheck scan

Rename atoms in the MSE residues SE->SD

In [ ]:
scanRenameAtoms scan { { :MSE { :SE :SD } } }

In [ ]:
scanAmberCheck scan

Remove the given atoms from the MLY residue.

In [ ]:
scanIgnoreAtoms scan { { :MLY { :HH22 :HH21 :HH23 :HH13 :HH12 :HH11 :CH2 :CH1} } }

Run a final check

In [ ]:
scanAmberCheck scan

## We now load an Amber compatible PDB file using the scan and the fixups that we just added.

In [ ]:
complex = loadPdb scan

In [ ]:
(show complex :axes t)

In [ ]:
alignAxes complex

In [ ]:
show complex :axes t

In [ ]:
setBox complex "vdw"

In [ ]:
(chem:geometric-center complex)

In [ ]:
show complex

In [ ]:
startSwank

In [ ]:
resolvateBox complex TIP3PBOX 8.0

In [ ]:
show complex

In [ ]:
ion-complex = copy complex

In [ ]:
addIons ion-complex NA 0

In [ ]:
addIons ion-complex CL 0

In [ ]:
saveamberparm ion-complex data/bsla/full_N_bsla_spytag_helix_solv12A.prmtop data/bsla/full_N_bsla_spytag_helix_solv12A.inpcrd

Save the AMBER compatible PDB file.

In [ ]:
savepdb ion-complex data/bsla/full_N_bsla_spytag_helix_solv12A.pdb

## All done